In [9]:
import torch
import torch.nn as nn
bs,T = 2,3  # 批大小，输入序列长度
input_size, hidden_size = 4, 3  # 输入特征大小，赠含展特任大么
input = torch.randn (bs, T, input_size)  # 随机初始化一八输人特征序列
h_prev = torch.zeros (bs, hidden_size)  #初始隐含状态
#  step 1 #PyTorch RNN API
rnn = nn.RNN (input_size, hidden_size, batch_first=True)
rnn_output, state_final = rnn (input, h_prev.unsqueeze(0))
# print (rnn output)
# print (state final)
# step2手写一个rnn forward西数，实现RNN的计簿原理
def rnn_forward(input, weight_ih, weight_hh, bias_ih, bias_hh, h_prev):
    bs, T, input_size = input.shape
    h_dim = weight_ih.shape[0]
    h_out = torch.zeros(bs, T, h_dim) #初始化一个输出(状态)矩阵
    for t in range(T):
        x = input[:, t, :].unsqueeze(2)  # 获欣当前时刻翰人特征
        w_ih_batch = weight_ih.unsqueeze(0).tile(bs, 1, 1)
        w_hh_batch = weight_hh.unsqueeze(0).tile(bs, 1, 1)
        
        w_times_x = torch.bmm(w_ih_batch, x).squeeze(-1)
        w_times_h = torch.bmm(w_hh_batch, h_prev.unsqueeze(2)).squeeze(-1)
        h_prev = torch.tanh(w_times_x+bias_ih+w_times_h+bias_hh)
        
        h_out[:, t, :] = h_prev
    return h_out, h_prev.unsqueeze(0)

In [10]:
for k, v in rnn.named_parameters():
    print(k, v)

weight_ih_l0 Parameter containing:
tensor([[-0.0912, -0.3464, -0.2707,  0.0140],
        [-0.1040,  0.3496, -0.4886, -0.1607],
        [ 0.2690,  0.3982, -0.5413,  0.1609]], requires_grad=True)
weight_hh_l0 Parameter containing:
tensor([[-0.0776, -0.1590, -0.1580],
        [-0.4207,  0.5501,  0.0582],
        [ 0.4584,  0.2644,  0.0811]], requires_grad=True)
bias_ih_l0 Parameter containing:
tensor([ 0.4935, -0.4461, -0.2190], requires_grad=True)
bias_hh_l0 Parameter containing:
tensor([ 0.0064,  0.0208, -0.4726], requires_grad=True)


In [12]:
custom_rnn_output, custom_state_final = rnn_forward(input, rnn.weight_ih_l0, rnn.weight_hh_l0, rnn.bias_ih_l0, rnn.bias_hh_l0, h_prev)


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import string

# Define the character set and mapping dictionaries
chars = string.ascii_lowercase + string.ascii_uppercase + string.punctuation + ' '
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
idx_to_char = {idx: ch for idx, ch in enumerate(chars)}

# Model definition
class RNNLayerNorm(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, nonlinearity='tanh', 
                 bias=True, batch_first=True, dropout=0.0, bidirectional=False):
        super(RNNLayerNorm, self).__init__()
        
        # RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, nonlinearity=nonlinearity,
                          bias=bias, batch_first=batch_first, dropout=dropout, 
                          bidirectional=bidirectional)
        
        # Layer normalization
        self.layer_norm = nn.LayerNorm(hidden_size)
        
        # Fully connected layer for output prediction
        self.fc = nn.Linear(hidden_size, input_size)
    
    def forward(self, x, h0=None):
        # Forward pass through RNN
        rnn_out, hn = self.rnn(x, h0)
        
        # Apply layer normalization to the output of the RNN
        norm_out = self.layer_norm(rnn_out)
        
        # Output layer
        output = self.fc(norm_out)
        
        return output, hn

# Helper functions
def encode_input(text):
    return torch.tensor([[char_to_idx[ch] for ch in text]], dtype=torch.float32)

def decode_output(indices):
    return ''.join([idx_to_char[idx] for idx in indices])

# Parameters
input_size = len(chars)
hidden_size = 128
num_layers = 2
batch_first = True
warmup_text = "Hello, how do"
num_predictions = 5

# Create the model
model = RNNLayerNorm(input_size, hidden_size, num_layers, batch_first=batch_first)
model.eval()

# Generate one-hot encoding for warm-up input
warmup_input = encode_input(warmup_text)
one_hot_input = F.one_hot(warmup_input.long(), num_classes=input_size).float()

# Initialize hidden state
h0 = torch.zeros(num_layers, 1, hidden_size)

# Warm-up phase
with torch.no_grad():
    for i in range(one_hot_input.size(1)):
        input_char_tensor = one_hot_input[:, i, :].unsqueeze(1)
        _, h0 = model(input_char_tensor, h0)

# Predict next tokens
predicted_text = warmup_text
input_char = warmup_text[-1]

for _ in range(num_predictions):
    input_tensor = encode_input(input_char).unsqueeze(0)
    one_hot_input = F.one_hot(input_tensor.long(), num_classes=input_size).float()
    
    with torch.no_grad():
        output, h0 = model(one_hot_input, h0)
    
    predicted_idx = torch.argmax(output[:, -1, :], dim=-1).item()
    predicted_char = idx_to_char[predicted_idx]
    predicted_text += predicted_char
    
    input_char = predicted_char

print("Predicted text:", predicted_text)


ValueError: RNN: Expected input to be 2D or 3D, got 4D tensor instead